In [ ]:
pip install alice_blue

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 8.3 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 24 13:47:17 2020

@author: Harish Rajakanthan

"""
import logging
import sys
import pytz
import datetime
#logging.basicConfig(level=logging.ERROR)
from alice_blue import *
import psutil
import time
##from telegram_send import telegram_bot_sendtext

#=============================================================================
# Initialize current date and time in IST
#=============================================================================

ist = pytz.timezone('Asia/Calcutta')
curr_time=datetime.datetime.now(ist)
print('Program start time in IST =', curr_time)
curr_date=str(curr_time).split(' ')[0]

#=============================================================================
# Backup incase futures retrieval fails
#=============================================================================
curr_month = curr_time.strftime("%b").upper()
ins_usd = "USDINR" + " " + curr_month + " FUT"
ins_bn  = "BANKNIFTY" + " " + curr_month + " FUT"
ins_n   = "NIFTY" + " " + curr_month + " FUT"
#=============================================================================
# File names
#=============================================================================
out_file="/home/harishh48/alice/output/" + "output_" + curr_date + ".csv"
hol_file="/home/harishh48/alice/bin/nse_holidays.txt"

#=============================================================================
#Check for NSE Holidays
#=============================================================================
#holiday_file = open(hol_file, "r")
#holiday_list=[]
#for h in holiday_file:
#    holiday_list.append(h.strip())
#
#if curr_date in holiday_list:
#    msg=curr_date + ": Today is a nse holiday"
#    print(msg)
#    send_tele=telegram_bot_sendtext(msg)
#    sys.exit()

#=============================================================================
#Get access token with credentials
#=============================================================================
username    = ''
password    = ''
twoFA       = ''
app_id      = "gyjkltyulr6ul"
api_secret  = "XsXAb6mTkx1KIhIXgyutyultulKyGxxN6z7mdHvmtT8jmZvxYP0AM5C8nBUAukb59Sscf"

#client_id = 'UNOFFICED'
#client_secret = 'AYBTHY25UFEDJ6EZY6ZISK1K0LBEUX18XGKB038P8RJF2660WHAZNWP07025XXL6'
#redirect_url= 'https://www.unofficed.com/alice/fallback/'

session_id = AliceBlue.login_and_get_sessionID(username=username,password=password,twoFA=twoFA,app_id=app_id,api_secret=api_secret)
#access_token = AliceBlue.login_and_get_access_token(username=username, password=password, twoFA=twoFA,  api_secret=client_secret, redirect_url=redirect_url, app_id=client_id)
alice = AliceBlue(username = username, session_id = session_id, master_contracts_to_download=['CDS','NSE','NFO'])
#alice = AliceBlue(username=username, password=password, access_token=access_token, master_contracts_to_download=['CDS','NSE','NFO'])

#=============================================================================
# Get the current future symbol for USDINR
#=============================================================================
def search_latest_expiry(exch,instr):
    min_expiry = datetime.datetime.strptime('2100-01-01', '%Y-%m-%d').date()
    all_scrips = alice.search_instruments(exch,instr)
    expiry_list=[]

    for scrip in all_scrips:
        symbol = str(scrip[2])
        expiry = scrip[4]
        #expire =
        if "CE" not in symbol and "PE" not in symbol and str(expiry) != "None" and symbol.startswith(instr):
            expiry_list.append(str(expiry)+":"+symbol)
            if expiry < min_expiry:
                min_expiry = expiry
                ins = symbol

    if str(curr_date) == str(min_expiry):
        expiry_list.sort()
        ins = str(expiry_list[1]).split(":")[1]

    return ins

ins_usd = search_latest_expiry('CDS', 'USDINR')
ins_bn = search_latest_expiry('NFO', 'BANKNIFTY')
ins_n = search_latest_expiry('NFO', 'NIFTY')
print("Current expiry for USDINR is ",ins_usd)
print("Current expiry for BANKNIFTY is ",ins_bn)
print("Current expiry for NIFTY is ",ins_n)

#=============================================================================

#=============================================================================
#Initial sleep time till 9:AM / Job starts at 8:55AM
#=============================================================================
start_time=datetime.datetime.now(ist).replace(hour=9, minute=0, second=0, microsecond=0)
curr_time=datetime.datetime.now(ist)
while curr_time < start_time:
    print("Waiting for 9AM.....")
    time.sleep(5)
    curr_time=datetime.datetime.now(ist)
#=============================================================================

#print(alice.get_balance()) # get balance / margin limits
#print(alice.get_profile()) # get profile
#print(alice.get_daywise_positions()) # get daywise positions
#print(alice.get_netwise_positions()) # get netwise positions
#print(alice.get_holding_positions()) # get holding positions
#ins_usd="INFY AUG FUT"
#usdinr_fut = alice.get_instrument_for_fno(symbol = ins_usd, expiry_date=datetime.date(2020, 8, 27), is_fut=True, strike=None, is_CE = False)

#=============================================================================
# Start action
#=============================================================================
##op_file = open(out_file, "a")
socket_opened = False
def event_handler_quote_update(message):
#    print(f"quote update {message}")
    global ticks
    ticks = message
    ##msg = str(ticks['instrument'][2])+","+str(ticks['instrument'][4])+","+str(ticks['exchange_time_stamp'])+","+str(ticks['ltt'])+","+str(ticks['ltp'])+","+str(ticks['best_bid_price'])+","+str(ticks['best_ask_price'])+","+str(ticks['total_buy_quantity'])+","+str(ticks['total_sell_quantity'])+","+str(ticks['open'])+","+str(ticks['high'])+","+str(ticks['low'])+","+str(ticks['close'])+","+str(ticks['atp'])+","+str(ticks['volume'])
    print(str(ticks))
    ##print(ticks)
    ##op_file.write(msg + "\n")
    #op_file.write(str(ticks))
    #op_file.write(str(ticks['instrument'][2]))
    #print(ticks['token'])
    #print(ticks['ltp'])

#def open_callback():
    #global socket_opened
    #socket_opened = True

#alice.start_websocket(subscribe_callback=event_handler_quote_update,
#                      socket_open_callback=open_callback,
#                      run_in_background=True)

alice.start_websocket(subscribe_callback=event_handler_quote_update)

#while(socket_opened==False):
#    pass
#all_sensex_scrips = alice.search_instruments('NSE', 'as')
#print(all_sensex_scrips)

#alice.subscribe([alice.get_instrument_by_symbol('CDS',ins_usd),alice.get_instrument_by_symbol('NSE',"Nifty Bank"),alice.get_instrument_by_symbol('NSE',"Nifty 50"),alice.get_instrument_by_symbol('NSE',"India VIX"),alice.get_instrument_by_symbol('NFO',ins_bn),alice.get_instrument_by_symbol('NFO',ins_n)], LiveFeedType.MARKET_DATA)
alice.subscribe([alice.get_instrument_by_symbol('CDS',ins_usd),alice.get_instrument_by_symbol('NFO',ins_bn),alice.get_instrument_by_symbol('NFO',ins_n)], LiveFeedType.TICK_DATA)
x = True
nse_eod=False
while x == True:
    print("Fetching data....")
    time.sleep(900)

    # gives a single float value
    print("CPU percentage: ",psutil.cpu_percent())
    # gives an object with many fields
    print("Virtual memory: ",psutil.virtual_memory()._asdict())
    # you can have the percentage of used RAM
    print("Percentage of used RAM: ",psutil.virtual_memory().percent)
    # you can calculate percentage of available memory
    print("Percentage of available memory: ",psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)

    print("Subscribed instruments are ....")
    print(alice.get_all_subscriptions())

    eod_time=datetime.datetime.now(ist).replace(hour=17, minute=0, second=0, microsecond=0)
    nse_eod_time=datetime.datetime.now(ist).replace(hour=15, minute=30, second=0, microsecond=0)
    curr_time=datetime.datetime.now(ist)
    print("Current time is :",curr_time)
    if curr_time > nse_eod_time and nse_eod == False:
        alice.unsubscribe([alice.get_instrument_by_symbol('NSE',"Nifty Bank"),alice.get_instrument_by_symbol('NSE',"Nifty 50"),alice.get_instrument_by_symbol('NSE',"India VIX"),alice.get_instrument_by_symbol('NFO',ins_bn),alice.get_instrument_by_symbol('NFO',ins_n)], LiveFeedType.MARKET_DATA)
        nse_eod = True
    if curr_time > eod_time:
        alice.unsubscribe([alice.get_instrument_by_symbol('CDS',ins_usd),alice.get_instrument_by_symbol('NSE',"Nifty Bank"),alice.get_instrument_by_symbol('NSE',"Nifty 50"),alice.get_instrument_by_symbol('NSE',"India VIX"),alice.get_instrument_by_symbol('NFO',ins_bn),alice.get_instrument_by_symbol('NFO',ins_n)], LiveFeedType.MARKET_DATA)
        op_file.close()
        x = False

print('Program end time in IST =', curr_time)
time.sleep(100)

Program start time in IST = 2022-10-08 12:13:25.698158+05:30
Current expiry for USDINR is  USDINR22OFUT
Current expiry for BANKNIFTY is  BANKNIFTY22OCTFUT
Current expiry for NIFTY is  NIFTY22OCTFUT


HTTPError: ignored